In [1]:
import pandas as pd
import os

In [3]:
#males
#45880669
#45878463

In [4]:
visit_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` visit 
        WHERE
        visit.PERSON_ID IN (
            select
                person_id  
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            where
                cb_search_person.person_id in (
                    select
                        person_id 
                    from
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    where
                        sex_at_birth_concept_id in (45880669) 
                ) 
            )"""

visit_df = pd.read_gbq(visit_sql, dialect="standard")
visit_df=visit_df.sort_values(by=['person_id','visit_start_datetime'])
visit_df.head(5)

Downloading: 100%|██████████| 4089601/4089601 [10:23<00:00, 6556.56rows/s]


,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitting_source_concept_id,admitting_source_value,discharge_to_concept_id,discharge_to_source_value,preceding_visit_occurrence_id
2516852,1000000664754777,1000000,9202,2018-09-18,2018-09-18 22:23:20+00:00,2018-09-18,2018-09-18 22:23:20+00:00,44818519,None,None,No matching concept,0,0,No matching concept,0,No matching concept,NaN
2569833,1000000975021357,1000004,9202,2019-04-08,2019-04-08 19:57:46+00:00,2019-04-08,2019-04-08 19:57:46+00:00,44818519,None,None,No matching concept,0,0,No matching concept,0,No matching concept,NaN
2636449,1000000146328718,1000033,9202,2018-09-11,2018-09-11 14:25:06+00:00,2018-09-11,2018-09-11 14:25:06+00:00,44818519,None,None,No matching concept,0,0,No matching concept,0,No matching concept,NaN
2580377,1000000311322817,1000042,9202,2018-11-12,2018-11-12 14:43:07+00:00,2018-11-12,2018-11-12 14:43:07+00:00,44818519,None,None,No matching concept,0,0,No matching concept,0,No matching concept,NaN
2784715,31000000122564619,1000079,9203,2018-04-14,2018-04-14 14:38:00+00:00,2018-04-14,2018-04-14 15:18:00+00:00,44818518,None,None,None,309310,309199,None,679545,None,NaN


In [5]:
visit_df=visit_df.sort_values(by=['person_id','visit_start_datetime'])

In [6]:
visit_df.head(5)

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitting_source_concept_id,admitting_source_value,discharge_to_concept_id,discharge_to_source_value,preceding_visit_occurrence_id
2516852,1000000664754777,1000000,9202,2018-09-18,2018-09-18 22:23:20+00:00,2018-09-18,2018-09-18 22:23:20+00:00,44818519,None,None,No matching concept,0,0,No matching concept,0,No matching concept,NaN
2569833,1000000975021357,1000004,9202,2019-04-08,2019-04-08 19:57:46+00:00,2019-04-08,2019-04-08 19:57:46+00:00,44818519,None,None,No matching concept,0,0,No matching concept,0,No matching concept,NaN
2636449,1000000146328718,1000033,9202,2018-09-11,2018-09-11 14:25:06+00:00,2018-09-11,2018-09-11 14:25:06+00:00,44818519,None,None,No matching concept,0,0,No matching concept,0,No matching concept,NaN
2580377,1000000311322817,1000042,9202,2018-11-12,2018-11-12 14:43:07+00:00,2018-11-12,2018-11-12 14:43:07+00:00,44818519,None,None,No matching concept,0,0,No matching concept,0,No matching concept,NaN
2784715,31000000122564619,1000079,9203,2018-04-14,2018-04-14 14:38:00+00:00,2018-04-14,2018-04-14 15:18:00+00:00,44818518,None,None,None,309310,309199,None,679545,None,NaN


In [7]:
visit_df=visit_df[['visit_occurrence_id', 'person_id',
       'visit_start_datetime', 'visit_end_datetime',
       'preceding_visit_occurrence_id']]

In [8]:
visit_df.to_csv('visitMale.csv',index=False)

In [9]:
visit_df.shape

(4089601, 5)

In [10]:
len(list(visit_df['visit_occurrence_id'].unique()))

4089601

In [11]:
len(list(visit_df['person_id'].unique()))

72573

In [13]:
visit_df = pd.read_csv('visitMale.csv',header=0)

In [36]:
visit_df=visit_df.sort_values(by=['person_id','visit_start_datetime'])

In [28]:
visit_df.to_csv('visitMale.csv',index=False)

In [20]:
#Calculate age at each visit
from datetime import datetime
visit_df['visit_start_datetime'] = pd.to_datetime(visit_df['visit_start_datetime'])
visit_df['birth_datetime'] = pd.to_datetime(visit_df['birth_datetime'])

In [21]:
visit_df['age']=visit_df['visit_start_datetime'] - visit_df['birth_datetime']

In [51]:
visit_df[['age']].head()

,age
0,790.526784
1,107.706944
2,111.573611
3,125.440278
4,132.306944


In [23]:
#Convert age to years
visit_df['age']=(((visit_df['age'].dt.total_seconds()/3600)/24)/30)

In [4]:
visit_df[['age']].head()

,age
0,65.877232
1,8.975579
2,9.297801
3,10.453356
4,11.025579


In [21]:
visit_df['age']=visit_df['age']/12

In [ ]:
#############MEAS##############

In [2]:
meas_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.measurement` meas 
        WHERE
        (
            meas.measurement_concept_id IN (
                40762636, 3004249, 3027018, 3012888, 3038553, 3036277, 3025315, 3000483
            ) 
            OR  meas.measurement_source_concept_id IN (
                40762636, 3004249, 3027018, 3012888, 3038553, 3036277, 3025315, 3000483
            )
        )  and
        meas.PERSON_ID IN (
            select
                person_id  
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            where
                cb_search_person.person_id in (
                    select
                        person_id 
                    from
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    where
                        sex_at_birth_concept_id in (45880669) 
                ) 
            )"""

meas_df = pd.read_gbq(meas_sql, dialect="standard")
meas_df.head(5)

Downloading: 100%|██████████| 11780685/11780685 [25:13<00:00, 7785.17rows/s]


,measurement_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,unit_concept_id,range_low,range_high,provider_id,visit_occurrence_id,measurement_source_value,measurement_source_concept_id,unit_source_value,value_source_value
0,38000000003810039,1698282,3004249,2016-10-02,2016-10-02 07:00:00+00:00,44818702,0.0,108.0,0.0,8876.0,0.0,0.0,None,3.800000e+16,No matching concept,0.0,mm[Hg],No matching concept
1,38000000008717310,1701004,3012888,2018-02-13,NaT,44818702,0.0,75.0,0.0,8876.0,0.0,0.0,None,3.800000e+16,No matching concept,0.0,mm[Hg],No matching concept
2,38000000002554647,1699677,3012888,2012-10-18,NaT,44818702,0.0,81.0,0.0,8876.0,0.0,0.0,None,3.800000e+16,No matching concept,0.0,mm[Hg],No matching concept
3,38000000002554840,1699677,3004249,2012-10-20,2012-10-20 22:12:00+00:00,44818702,0.0,125.0,0.0,8876.0,0.0,0.0,None,3.800000e+16,No matching concept,0.0,mm[Hg],No matching concept
4,38000000002719534,1667526,3004249,2014-01-29,2014-01-29 10:47:00+00:00,44818702,0.0,99.0,0.0,8876.0,0.0,0.0,None,3.800000e+16,No matching concept,0.0,mm[Hg],No matching concept


In [3]:
meas_df.shape

(11780685, 18)

In [4]:
meas_df = meas_df[meas_df['value_as_number'].notnull()]

In [5]:
meas_df=meas_df.sort_values(by=['person_id','visit_occurrence_id'])

In [6]:
meas_df.shape

(10771106, 18)

In [7]:
meas_df.to_csv('measM.csv',index=False)

In [10]:
meas_df=pd.merge(meas_df,visit_df[['person_id','visit_occurrence_id','age']],on=['person_id','visit_occurrence_id'],how='left')

In [11]:
meas_df.columns

Index(['measurement_id', 'person_id', 'measurement_concept_id',
       'measurement_date', 'measurement_datetime',
       'measurement_type_concept_id', 'operator_concept_id', 'value_as_number',
       'value_as_concept_id', 'unit_concept_id', 'range_low', 'range_high',
       'provider_id', 'visit_occurrence_id', 'measurement_source_value',
       'measurement_source_concept_id', 'unit_source_value',
       'value_source_value', 'visit_start_datetime', 'visit_end_datetime',
       'preceding_visit_occurrence_id', 'age'],
      dtype='object')

In [8]:
meas_df=meas_df[['measurement_id', 'person_id', 'measurement_concept_id',
        'value_as_number', 'visit_occurrence_id']]

In [25]:
meas_df.to_csv('measM.csv',index=False)

In [8]:
#####SELECT MEAS Cohort##############

In [10]:
cond_df=pd.read_csv('condM.csv',header=0)

In [14]:
cond_df.head()

,person_id,visit_occurrence_id,age,condition_concept_id,concept_name
0,1497223,4.600000e+16,81.362269,321588.0,Heart disease
1,1476700,1.100000e+16,65.187122,321588.0,Heart disease
2,1497223,4.600000e+16,81.487616,44784217.0,Cardiac arrhythmia
3,1497223,4.600000e+16,82.634693,44784217.0,Cardiac arrhythmia
4,1497223,4.600000e+16,81.362269,44784217.0,Cardiac arrhythmia


In [12]:
del cond_df['condition_concept_id']
del cond_df['concept_name']

In [13]:
cond_df = cond_df.rename(columns={'ancestor_id3':'condition_concept_id'})
cond_df = cond_df.rename(columns={'ancestor_name3':'concept_name'})

In [16]:
meas_df=pd.read_csv('measM.csv',header=0)

In [17]:
meas_df.head()

,measurement_id,person_id,measurement_concept_id,value_as_number,visit_occurrence_id
0,1000787374207007,1000091,3004249,117.000000,1.000001e+15
1,1000787374207001,1000091,3004249,125.000000,1.000001e+15
2,1000787374207011,1000091,3027018,60.000000,1.000001e+15
3,1000787374207015,1000091,3036277,161.800003,1.000001e+15
4,1000787374207004,1000091,3004249,121.000000,1.000001e+15


In [18]:
len(list(meas_df['person_id'].unique()))

113681

In [19]:
meas_df=meas_df[meas_df['person_id'].isin(list(cond_df['person_id'].unique()))]

In [20]:
len(list(meas_df['person_id'].unique()))

23021

In [21]:
meas_df=meas_df[meas_df['visit_occurrence_id'].isin(list(cond_df['visit_occurrence_id'].unique()))]

In [22]:
len(list(meas_df['person_id'].unique()))

17976

In [23]:
len(list(meas_df['visit_occurrence_id'].unique()))

200475

In [24]:
len(list(cond_df['visit_occurrence_id'].unique()))

470954

In [23]:
#####Done selecting cohort###################

In [ ]:
###############COND################

In [2]:
cond_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` cond 
        WHERE
        cond.PERSON_ID IN (
            select
                person_id  
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            where
                cb_search_person.person_id in (
                    select
                        person_id 
                    from
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    where
                        sex_at_birth_concept_id in (45880669) 
                ) 
            )"""

cond_df = pd.read_gbq(cond_sql, dialect="standard")
cond_df.head(5)

Downloading: 100%|██████████| 16132747/16132747 [46:52<00:00, 5735.31rows/s]


,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,stop_reason,provider_id,visit_occurrence_id,condition_source_value,condition_source_concept_id,condition_status_source_value,condition_status_concept_id
0,11000000124076176,1474841,312327,2014-03-13,2014-03-13 09:31:00.100780+00:00,NaT,NaT,44786627,None,None,1.100000e+16,410.90,44835928.0,89100005,4230359
1,11000000112219182,1474841,312327,2014-08-09,2014-08-09 16:23:00.101140+00:00,NaT,NaT,44786627,None,None,1.100000e+16,410.90,44835928.0,89100005,4230359
2,11000000128814265,1474841,312327,2014-03-06,2014-03-06 09:22:00.100700+00:00,NaT,NaT,44786627,None,None,1.100000e+16,410.90,44835928.0,89100005,4230359
3,11000000152666815,1474841,312327,2014-08-28,2014-08-28 09:30:00.101340+00:00,NaT,NaT,44786627,None,None,1.100000e+16,410.90,44835928.0,89100005,4230359
4,25000000004351992,1518849,4185932,2014-02-24,2014-02-24 05:00:00+00:00,2014-02-24,NaT,44786627,None,None,2.508000e+16,411.89,44820863.0,89100005,4230359


In [3]:
cond_df.shape

(16132747, 15)

In [ ]:
cond_df.to_csv('condM.csv',index=False)

In [2]:
cond_df=pd.read_csv('condM.csv',header=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cond_df.shape

(16132747, 15)

In [4]:
cond_df.head()

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,stop_reason,provider_id,visit_occurrence_id,condition_source_value,condition_source_concept_id,condition_status_source_value,condition_status_concept_id
0,11000000124076176,1474841,312327,2014-03-13,2014-03-13 09:31:00.100780+00:00,NaN,NaN,44786627,NaN,NaN,1.100000e+16,410.90,44835928.0,89100005,4230359.0
1,11000000112219182,1474841,312327,2014-08-09,2014-08-09 16:23:00.101140+00:00,NaN,NaN,44786627,NaN,NaN,1.100000e+16,410.90,44835928.0,89100005,4230359.0
2,11000000128814265,1474841,312327,2014-03-06,2014-03-06 09:22:00.100700+00:00,NaN,NaN,44786627,NaN,NaN,1.100000e+16,410.90,44835928.0,89100005,4230359.0
3,11000000152666815,1474841,312327,2014-08-28,2014-08-28 09:30:00.101340+00:00,NaN,NaN,44786627,NaN,NaN,1.100000e+16,410.90,44835928.0,89100005,4230359.0
4,25000000004351992,1518849,4185932,2014-02-24,2014-02-24 05:00:00+00:00,2014-02-24,NaN,44786627,NaN,NaN,2.508000e+16,411.89,44820863.0,89100005,4230359.0


In [5]:
len(list(cond_df['person_id'].unique()))

71121

In [ ]:
##############Combine with visit data to get age####################

In [6]:
visit=pd.read_csv('visitMale.csv',header=0)

In [7]:
visit.head()

,visit_occurrence_id,person_id,visit_start_datetime,visit_end_datetime,preceding_visit_occurrence_id,age
0,1000000787374207,1000091,2018-11-28 19:17:03+00:00,2018-11-28 19:17:03+00:00,NaN,790.526784
1,25080000095431272,1000093,2008-11-17 05:00:00+00:00,NaN,NaN,107.706944
2,25080000092412028,1000093,2009-03-13 05:00:00+00:00,NaN,NaN,111.573611
3,25080000086507289,1000093,2010-05-03 05:00:00+00:00,NaN,NaN,125.440278
4,25080000076363926,1000093,2010-11-25 05:00:00+00:00,NaN,NaN,132.306944


In [8]:
cond_df=pd.merge(cond_df,visit[['visit_occurrence_id', 'person_id','age']],how='left',on=['person_id','visit_occurrence_id'])

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:1089: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


In [9]:
cond_df = cond_df[cond_df['age'].notna()]

In [10]:
cond_df.shape

(7209718, 16)

In [11]:
len(list(cond_df['person_id'].unique()))

54781

In [12]:
cond_df.to_csv('condM.csv',index=False)

In [13]:
len(list(cond_df['condition_concept_id'].unique()))

18347

In [ ]:
#########Code Statistics##############

In [20]:
ncond = (cond_df[['condition_concept_id','person_id','visit_occurrence_id']].groupby(['condition_concept_id']).count()).reset_index()

In [21]:
ncond['condition_concept_id'].max()

46286594

In [22]:
ncond['condition_concept_id'].mean()

7965424.845914863

In [23]:
ncond['condition_concept_id'].min()

0

In [24]:
######Number of condition codes above mean##################
ncond[ncond['condition_concept_id']>7760039].shape

(2638, 3)

In [25]:
######Number of condition codes below mean##################
ncond[ncond['condition_concept_id']<=7760039].shape

(15709, 3)

In [26]:
#####Population loevel nCond###########
ncond=cond_df[['condition_concept_id','person_id']].groupby(['condition_concept_id','person_id']).size().reset_index(name='counts')

In [27]:
ncond.head()

,condition_concept_id,person_id,counts
0,0,1000104,28
1,0,1000109,13
2,0,1001161,13
3,0,1001290,1
4,0,1001591,31


In [28]:
#####Population loevel nCond###########
ncond=ncond[['condition_concept_id','person_id']].groupby(['condition_concept_id']).size().reset_index(name='counts')

In [29]:
ncond.head()

,condition_concept_id,counts
0,0,6759
1,22274,3
2,22281,28
3,22288,5
4,22340,96


In [30]:
ncond['counts'].max()

18553

In [31]:
ncond['counts'].mean()

96.59028724042078

In [34]:
######Number of condition codes above mean##################
ncond[ncond['counts']>96].shape

(2342, 2)

In [35]:

######Number of condition codes below mean##################
ncond[ncond['counts']<=96].shape

(16005, 2)

In [36]:
acids = ncond[ncond['counts']>96]['condition_concept_id']
bcids = ncond[ncond['counts']<=96]['condition_concept_id']

In [ ]:
cond_dfa=cond_df[cond_df['condition_concept_id'].isin(acids)]

In [ ]:
cond_dfa=cond_dfa[['person_id', 'condition_concept_id','visit_occurrence_id','age','concept_name']]

In [ ]:
###########CONCEPT_RELATIONSHIP#################

In [2]:
rel_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.concept_relationship` cond """

rel_df = pd.read_gbq(rel_sql, dialect="standard")
rel_df.head(5)

Downloading: 100%|██████████| 40886644/40886644 [43:32<00:00, 15651.10rows/s]


,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,4297957,46084219,Disp dose form of,2018-07-31,2099-12-31,None
1,35626985,46210804,Legal category of,2018-07-31,2099-12-31,None
2,4132161,46050651,Licensed route of,2015-01-31,2099-12-31,None
3,4294612,4153461,Disp dose form of,2015-01-31,2099-12-31,None
4,21600747,21061584,Drug class of drug,2015-08-17,2099-12-31,None


In [3]:
rel_df.columns

Index(['concept_id_1', 'concept_id_2', 'relationship_id', 'valid_start_date',
       'valid_end_date', 'invalid_reason'],
      dtype='object')

In [4]:
rel_df=rel_df[['concept_id_1', 'concept_id_2', 'relationship_id']]

In [5]:
rel_df.to_csv('rel.csv',index=False)

In [2]:
###### GROUP CONDS #######

In [104]:
rel_df=pd.read_csv('rel.csv',header=0)

In [105]:
rel_df.head()

,concept_id_1,concept_id_2
0,2847686,2801516
1,46052272,45968758
2,46204759,46091834
3,2752170,2860937
4,43020006,2806114


In [3]:
concept_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` cond 
    where
                    concept_id in (195083,194755,4109084,4182562)"""

concept_df = pd.read_gbq(concept_sql, dialect="standard")
concept_df


Downloading: 100%|██████████| 4/4 [00:00<00:00, 43.15rows/s]


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,4109084,Left sided abdominal pain,Condition,SNOMED,Clinical Finding,S,285387005,1970-01-01,2099-12-31,None
1,194755,Abdominal tenderness of left lower quadrant,Condition,SNOMED,Clinical Finding,S,439774009,2009-01-31,2099-12-31,None
2,195083,Left lower quadrant pain,Condition,SNOMED,Clinical Finding,S,301716002,1970-01-01,2099-12-31,None
3,4182562,Lower abdominal pain,Condition,SNOMED,Clinical Finding,S,54586004,1970-01-01,2099-12-31,None


In [ ]:
concept_df.to_csv('concept.csv',index=False)

In [53]:
#########Another Method#############

In [258]:
rel_df=rel_df[rel_df['relationship_id']=='Is a']

In [106]:
rel_df.columns

Index(['concept_id_1', 'concept_id_2'], dtype='object')

In [260]:
rel_df=rel_df[['concept_id_1', 'concept_id_2']]

In [113]:
concept=pd.read_csv('concept.csv',header=0)

In [108]:
concept=concept[concept['concept_name'].isin(['General finding of observation of patient','Finding of body region','Disorder of body system','Inflammation of specific body organs','Finding by site','Complication','Disease','Clinical finding','Abdominal organ finding','Acute disease','Chronic disease','Choroidal and/or chorioretinal disorder','Evaluation finding','Disorder by body site','Measurement finding','Measurement finding below reference range','Measurement finding outside reference range','SNOMED CT Concept','Clinical history and observation findings','Degenerative disorder','Disease related state','Disorder in full remission','Finding by method','Functional finding','General body state finding'])]

In [109]:
rel_df=rel_df[~rel_df['concept_id_2'].isin(list(concept['concept_id']))]

In [111]:
concept.head()

,concept_id,concept_name
387139,4137797,Complication
452965,45612949,Disease
504665,45885572,Chronic disease
517856,4008453,SNOMED CT Concept
970511,36308340,Disease


In [ ]:
concept=pd.read_csv('concept.csv',header=0)

In [83]:
concept=concept.rename(columns={'concept_id':'condition_concept_id'})

In [84]:
cond_dfa=pd.merge(cond_dfa,concept,how='left',on='condition_concept_id')

In [85]:
concept=concept.rename(columns={'condition_concept_id':'concept_id'})

In [112]:
rel_df=rel_df[rel_df['concept_id_1']!=43530680]

In [114]:
rel_df=rel_df.groupby('concept_id_1').first().reset_index()

In [117]:
rel_df.head()

,concept_id_1,concept_id,concept_name
0,5084,8756,Outpatient Hospital
1,8537,581476,Home Visit
2,8546,42898160,Non-hospital institution Visit
3,8584,9202,Outpatient Visit
4,8602,581476,Home Visit


In [116]:
rel_df=rel_df.rename(columns={'concept_id_2':'concept_id'})
rel_df=pd.merge(rel_df, concept, on='concept_id',how='left')

In [102]:
len(list(cond_dfa['person_id'].unique()))

54335

In [119]:
rel_df.head()

,condition_concept_id,ancestor_id,ancestor_name
0,5084,8756,Outpatient Hospital
1,8537,581476,Home Visit
2,8546,42898160,Non-hospital institution Visit
3,8584,9202,Outpatient Visit
4,8602,581476,Home Visit


In [118]:
rel_df=rel_df.rename(columns={'concept_id_1':'condition_concept_id'})
rel_df=rel_df.rename(columns={'concept_id':'ancestor_id'})
rel_df=rel_df.rename(columns={'concept_name':'ancestor_name'})

In [120]:
cond_dfa=cond_dfa.fillna(0)

In [121]:
cond_dfa=cond_dfa[cond_dfa['condition_concept_id']!=0]

In [122]:
cond_dfa=pd.merge(cond_dfa, rel_df, on='condition_concept_id',how='left')

In [123]:
len(list(cond_dfa['condition_concept_id'].unique()))

2341

In [129]:
len(list(cond_dfa['ancestor_id'].unique()))

1630

In [126]:
cond_dfa=cond_dfa.fillna(0)

In [128]:
cond_dfa.loc[cond_dfa['ancestor_id']==0,'ancestor_id']=cond_dfa.loc[cond_dfa['ancestor_id']==0,'condition_concept_id']
cond_dfa.loc[cond_dfa['ancestor_name']==0,'ancestor_name']=cond_dfa.loc[cond_dfa['ancestor_name']==0,'concept_name']

In [42]:
cond_dfa=pd.read_csv('cond_dfa.csv',header=0)

In [131]:
rel_df=rel_df.rename(columns={'ancestor_id':'ancestor_id2'})
rel_df=rel_df.rename(columns={'ancestor_name':'ancestor_name2'})
rel_df=rel_df.rename(columns={'condition_concept_id':'ancestor_id'})

In [132]:
rel_df.head()

,ancestor_id,ancestor_id2,ancestor_name2
0,5084,8756,Outpatient Hospital
1,8537,581476,Home Visit
2,8546,42898160,Non-hospital institution Visit
3,8584,9202,Outpatient Visit
4,8602,581476,Home Visit


In [133]:
cond_dfa=pd.merge(cond_dfa, rel_df, on=['ancestor_id'],how='left')

In [139]:
len(list(cond_dfa['ancestor_id2'].unique()))

1047

In [136]:
cond_dfa=cond_dfa.fillna(0)

In [138]:
cond_dfa.loc[cond_dfa['ancestor_id2']==0,'ancestor_id2']=cond_dfa.loc[cond_dfa['ancestor_id2']==0,'ancestor_id']
cond_dfa.loc[cond_dfa['ancestor_name2']==0,'ancestor_name2']=cond_dfa.loc[cond_dfa['ancestor_name2']==0,'ancestor_name']

In [140]:
rel_df=rel_df.rename(columns={'ancestor_id2':'ancestor_id3'})
rel_df=rel_df.rename(columns={'ancestor_name2':'ancestor_name3'})
rel_df=rel_df.rename(columns={'ancestor_id':'ancestor_id2'})

In [141]:
rel_df.head()

,ancestor_id2,ancestor_id3,ancestor_name3
0,5084,8756,Outpatient Hospital
1,8537,581476,Home Visit
2,8546,42898160,Non-hospital institution Visit
3,8584,9202,Outpatient Visit
4,8602,581476,Home Visit


In [142]:
cond_dfa=pd.merge(cond_dfa, rel_df, on=['ancestor_id2'],how='left')

In [149]:
len(list(cond_dfa['ancestor_id3'].unique()))

662

In [145]:
cond_dfa=cond_dfa.fillna(0)

In [147]:
cond_dfa.loc[cond_dfa['ancestor_id3']==0,'ancestor_id3']=cond_dfa.loc[cond_dfa['ancestor_id3']==0,'ancestor_id2']
cond_dfa.loc[cond_dfa['ancestor_name3']==0,'ancestor_name3']=cond_dfa.loc[cond_dfa['ancestor_name3']==0,'ancestor_name2']

In [148]:
len(list(cond_dfa['person_id'].unique()))

54335

In [150]:
cond_dfa.to_csv('cond_dfa.csv',index=False)

In [323]:
cond_dfa=cond_dfa.sort_values(['person_id','age'])

In [167]:
cond_dfa.columns

Index(['person_id', 'condition_concept_id', 'visit_occurrence_id', 'age',
       'concept_name', 'ancestor_id', 'ancestor_name', 'ancestor_id2',
       'ancestor_name2', 'ancestor_id3', 'ancestor_name3'],
      dtype='object')

In [168]:
cond_dfa=cond_dfa[['person_id', 'condition_concept_id', 'visit_occurrence_id', 'age',
       'concept_name','ancestor_id3', 'ancestor_name3']]

In [254]:
cond_dfa=cond_dfa.fillna(0)

In [ ]:
################End of Grouping##############

In [2]:
cond_dfa=pd.read_csv('cond_dfa.csv',header=0)

In [152]:
#max number of visits
(cond_dfa.groupby(['person_id'])['visit_occurrence_id'].count()).mean()

118.4769301555167

In [153]:
startVisit = cond_dfa.groupby(['person_id'])['age'].min()
endVisit = cond_dfa.groupby(['person_id'])['age'].max()

In [154]:
startVisit.reset_index()

,person_id,age
0,1000093,8.975579
1,1000104,38.606622
2,1000109,32.973302
3,1000174,26.161806
4,1000307,47.330556
...,...,...
54330,3524486,50.019147
54331,3524519,79.207207
54332,3524530,52.208333
54333,3524591,50.856418


In [155]:
endVisit.reset_index()

,person_id,age
0,1000093,17.521402
1,1000104,53.864988
2,1000109,47.048301
3,1000174,38.672917
4,1000307,52.177778
...,...,...
54330,3524486,50.019147
54331,3524519,85.923781
54332,3524530,66.711111
54333,3524591,57.206281


In [156]:
medLen = (endVisit - startVisit).reset_index()

In [157]:
medLen

,person_id,age
0,1000093,8.545824
1,1000104,15.258366
2,1000109,14.074998
3,1000174,12.511111
4,1000307,4.847222
...,...,...
54330,3524486,0.000000
54331,3524519,6.716574
54332,3524530,14.502778
54333,3524591,6.349863


In [158]:
medLen['age'].max()

38.91388888888889

In [159]:
medLen['age'].min()

0.0

In [160]:
medLen['age'].mean()

6.978344553071367

In [162]:
medLen=medLen[(medLen['age']>=2) & (medLen['age']<=10)]

In [163]:
pids=list(medLen['person_id'].unique())

In [164]:
cond_dfa=cond_dfa[cond_dfa['person_id'].isin(pids)]

In [165]:
len(list(cond_dfa['person_id'].unique()))

23074

In [169]:
cond_dfa.to_csv('condF.csv',index=False)

In [ ]:
############end of cohort selection################

In [26]:
drug_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.drug_exposure` drug 
        WHERE
        drug.PERSON_ID IN (
            select
                person_id  
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            where
                cb_search_person.person_id in (
                    select
                        person_id 
                    from
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    where
                        sex_at_birth_concept_id in (45880669) 
                ) 
            )"""

drug_df = pd.read_gbq(drug_sql, dialect="standard")
drug_df.head(5)

Downloading: 100%|██████████| 11809759/11809759 [31:21<00:00, 6276.78rows/s]


,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,...,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
0,11000000037568413,1428755,19003084,2006-11-21,2006-11-21 10:23:34.798202+00:00,NaT,NaT,NaT,38000177,None,...,NaN,None,4184451.0,None,None,NaN,None,NaN,54485002,None
1,35000000000705002,1422186,19074220,2017-05-14,2017-05-14 06:00:00+00:00,2018-02-04,2018-02-04 06:00:00+00:00,2018-02-04,38000177,None,...,NaN,None,4184451.0,None,None,3.500000e+16,None,NaN,54485002,None
2,24000000001801745,1417952,19076734,2016-07-12,2016-07-12 14:10:00+00:00,2016-07-12,2016-07-12 14:10:00+00:00,NaT,38000177,None,...,NaN,None,4184451.0,None,None,2.400000e+16,None,NaN,54485002,None
3,8000000000239342,1401613,45776321,2013-04-26,2013-04-26 15:21:00+00:00,2013-04-27,2013-04-27 03:10:00+00:00,NaT,38000177,None,...,NaN,None,4184451.0,None,None,8.000000e+15,None,NaN,54485002,None
4,24000000000645116,1451633,1594709,2015-12-07,2015-12-07 21:00:00+00:00,2015-12-10,2015-12-10 14:49:00+00:00,NaT,38000177,None,...,NaN,None,4184451.0,None,None,2.400000e+16,None,NaN,54485002,None


In [27]:
drug_df.shape

(11809759, 22)

In [29]:
drug_df=drug_df[['drug_exposure_id', 'person_id', 'drug_concept_id','visit_occurrence_id','drug_source_concept_id']]

In [38]:
drug_df.to_csv('drugF.csv',index=False)

In [46]:
drug_df=pd.read_csv('drugF.csv',header=0)

In [31]:
cond_df=pd.read_csv('condF.csv',header=0)

In [3]:
len(list(drug_df['person_id'].unique()))

20665

In [33]:
drug_df=drug_df[drug_df['person_id'].isin(list(cond_df['person_id'].unique()))]

In [34]:
len(list(drug_df['person_id'].unique()))

22743

In [35]:
drug_df=drug_df[drug_df['visit_occurrence_id'].isin(list(cond_df['visit_occurrence_id'].unique()))]

In [36]:
len(list(drug_df['person_id'].unique()))

20665

In [37]:
len(list(drug_df['person_id'].unique()))

20665

In [ ]:
######Remove below mean ids################

In [ ]:
ncond=drug_df[['drug_concept_id','person_id']].groupby(['drug_concept_id','person_id']).size().reset_index(name='counts')

In [ ]:
del ncond['counts']

In [ ]:
ncond=ncond[['drug_concept_id','person_id']].groupby(['drug_concept_id']).size().reset_index(name='counts')

In [ ]:
ncond['counts'].max()

In [ ]:
ncond['counts'].mean()

In [ ]:
acids = ncond[ncond['counts']>37]['drug_concept_id']

In [ ]:
drug_df=drug_df[drug_df['drug_concept_id'].isin(acids)]

In [ ]:
###########grouping drug###############

In [8]:
drug_df.head()

,drug_exposure_id,person_id,drug_concept_id,visit_occurrence_id,drug_source_concept_id
0,21000000000417687,3190258,0,2.100000e+16,0.0
1,38000000000237748,3201229,1112807,3.800000e+16,0.0
2,38000000002007437,3167819,797399,3.800000e+16,0.0
3,21000000000478158,3200713,0,2.100000e+16,0.0
4,38000000011649244,3146939,1560524,3.800000e+16,0.0


In [20]:
len(list(drug_df['drug_concept_id'].unique()))

12086

In [47]:
rel_df=pd.read_csv('rel.csv',header=0)

In [48]:
rel_df.columns

Index(['concept_id_1', 'concept_id_2', 'relationship_id'], dtype='object')

In [51]:
rel_df=rel_df.groupby('concept_id_1').first().reset_index()

In [52]:
rel_df.head()

,concept_id_1,concept_id_2
0,501343,21601262
1,501346,21601262
2,501347,21601262
3,501415,21601262
4,501416,21601262


In [24]:
drug_df.head()

,drug_exposure_id,person_id,drug_concept_id,visit_occurrence_id,drug_source_concept_id
0,8000000000239342,1401613,45776321,8.000000e+15,NaN
1,8000000000171460,1404427,906116,8.000000e+15,NaN
2,24000000000945505,1481998,19102200,2.400000e+16,NaN
3,8000000000207775,1404427,19102471,8.000000e+15,NaN
4,6000000106855320,1434269,19011661,6.000000e+15,19011661.0


In [49]:
rel_df=rel_df[rel_df['relationship_id']=='RxNorm - ATC']

In [50]:
rel_df=rel_df[['concept_id_1','concept_id_2']]

In [66]:
drug_df = drug_df[drug_df['drug_concept_id']!=0]

In [67]:
drug_df.shape

(1189946, 5)

In [68]:
len(list(drug_df['person_id'].unique()))

20151

In [69]:
rel_df=rel_df.rename(columns={'concept_id_1':'drug_concept_id'})
rel_df=rel_df.rename(columns={'concept_id_2':'ancestor1'})

In [70]:
drug_df=pd.merge(drug_df, rel_df, on='drug_concept_id',how='left')

In [71]:
len(list(drug_df['ancestor1'].unique()))

266

In [72]:
drug_df=drug_df.fillna(0)

In [73]:
drug_df.loc[drug_df['ancestor1']==0,'ancestor1']=drug_df.loc[drug_df['ancestor1']==0,'drug_concept_id']

In [74]:
len(list(drug_df['ancestor1'].unique()))

1283

In [75]:
rel_df=rel_df.rename(columns={'ancestor1':'ancestor2'})
rel_df=rel_df.rename(columns={'drug_concept_id':'ancestor1'})

In [76]:
drug_df=pd.merge(drug_df, rel_df, on='ancestor1',how='left')

In [77]:
(list(drug_df['ancestor2'].unique()))

[nan]

In [78]:
drug_df=drug_df.fillna(0)

In [79]:
drug_df.loc[drug_df['ancestor2']==0,'ancestor2']=drug_df.loc[drug_df['ancestor2']==0,'ancestor1']

In [80]:
len(list(drug_df['ancestor2'].unique()))

1283

In [81]:
drug_df.head()

,drug_exposure_id,person_id,drug_concept_id,visit_occurrence_id,drug_source_concept_id,ancestor1,ancestor2
0,8000000000239342,1401613,45776321,8.000000e+15,0.0,45776321.0,45776321.0
1,8000000000171460,1404427,906116,8.000000e+15,0.0,21605065.0,21605065.0
2,24000000000945505,1481998,19102200,2.400000e+16,0.0,19102200.0,19102200.0
3,8000000000207775,1404427,19102471,8.000000e+15,0.0,21603615.0,21603615.0
4,6000000106855320,1434269,19011661,6.000000e+15,19011661.0,19011661.0,19011661.0


In [82]:
drug_df.columns

Index(['drug_exposure_id', 'person_id', 'drug_concept_id',
       'visit_occurrence_id', 'drug_source_concept_id', 'ancestor1',
       'ancestor2'],
      dtype='object')

In [83]:
drug_df=drug_df[[ 'person_id', 'drug_concept_id',
       'visit_occurrence_id','ancestor1' ]]

In [84]:
drug_df.to_csv('drugFReduced.csv',index=False)

In [64]:
len(list(drug_df['person_id'].unique()))

20215

In [ ]:
####end of removing ids###########

In [85]:
concept=pd.read_csv('concept.csv',header=0)

In [86]:
concept.head()

,concept_id,concept_name
0,2102339,"Allograft skin for temporary wound closure, fa..."
1,42734947,"Intravascular radiopharmaceutical therapy, par..."
2,42735674,"Fasciotomy, lateral or medial (eg, tennis elbo..."
3,42739401,"Subsequent intensive care, per day, for the ev..."
4,2106743,"Biopsy, pleura; open"


In [87]:
concept=concept.rename(columns={'concept_id':'ancestor1'})

In [88]:
drug_df=pd.merge(drug_df,concept,how='left',on=['ancestor1'])

In [40]:
len(list(drug_df['concept_name'].unique()))

1279

In [89]:
drug_df.head()

,person_id,drug_concept_id,visit_occurrence_id,ancestor1,concept_name
0,1401613,45776321,8.000000e+15,45776321.0,Tetracaine hydrochloride 5 MG/ML Ophthalmic So...
1,1404427,906116,8.000000e+15,21605065.0,tropicamide
2,1481998,19102200,2.400000e+16,19102200.0,Cyclosporine 0.5 MG/ML Ophthalmic Suspension [...
3,1404427,19102471,8.000000e+15,21603615.0,MOXIFLOXACIN
4,1434269,19011661,6.000000e+15,19011661.0,Polyvinyl Alcohol 0.014 ML/ML Ophthalmic Solution


In [90]:
drug_df.to_csv('drugM.csv',index=False)